In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score


In [35]:
products = pd.read_csv(r'E:\Capstone DS\Recommender\product.csv')
customer = pd.read_csv(r'E:\Capstone DS\Recommender\customer.csv')
transactions = pd.read_csv(r'E:\Capstone DS\Recommender\transactions.csv')


In [36]:
data = transactions.merge(products, on='prod_cat_code').merge(customer, on='customer_id')

KeyError: 'customer_id'

In [29]:
data['prod_cat_code'] = data['prod_cat_code'].astype('category').cat.codes
data['customer_id'] = data['customer_id'].astype('category').cat.codes

NameError: name 'data' is not defined

In [ ]:
interaction_matrix = data.pivot_table(index='customer_id', columns='prod_cat_code', values='purchase', fill_value=0)

In [ ]:
train_data, test_data = train_test_split(interaction_matrix, test_size=0.2, random_state=42)

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(train_data)


In [ ]:
def recommend_products(customer_id, n_recommendations=5):
    distances, indices = model.kneighbors(train_data.loc[customer_id].values.reshape(1, -1), n_neighbors=n_recommendations+1)
    recommended_product_ids = [train_data.index[i] for i in indices.flatten()][1:]
    return recommended_product_ids

In [ ]:
customer_id = 1  # Replace with actual customer ID
recommended_products = recommend_products(customer_id)
print(f"Recommended products for customer {customer_id}: {recommended_products}")

In [16]:
def evaluate_model(test_data, model):
    y_true = []
    y_pred = []
    for customer_id in test_data.index:
        true_products = test_data.loc[customer_id][test_data.loc[customer_id] > 0].index.tolist()
        recommended_products = recommend_products(customer_id)
        
        y_true.append(true_products)
        y_pred.append(recommended_products)
    y_true = [item for sublist in y_true for item in sublist]
    y_pred = [item for sublist in y_pred for item in sublist]
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
map_score = average_precision_score(y_true, y_pred, average='macro')
    
return precision, recall, f1, map_score

precision, recall, f1, map_score = evaluate_model(test_data, model)
print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}, MAP: {map_score}")

NameError: name 'y_true' is not defined